In [104]:
import pandas as pd
import json
import numpy as np
import pickle
from struct import unpack
from base64 import b64decode
from functools import partial
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import torch.nn as nn

In [2]:
IMG_LEN = 1024
TXT_LEN = 300
N_TOPICS = 50
N_WORTHINESSES = 2

### Loading data

In [3]:
def unpck(l, x):
    return unpack('%df' % l, b64decode(x.encode('utf-8')))

In [10]:
unpck_img = partial(unpck, IMG_LEN)
unpck_txt = partial(unpck, TXT_LEN)
filename = "data/topics_worthiness_dataset.txt"

In [11]:
def get_df():
    return pd.read_json(filename, lines=True)

In [ ]:
with open('file.json') as f:
    ds_json = json.load(f)

In [12]:
df = get_df()

In [13]:
df.columns

Index(['fasttext_embed', 'image_embed', 'topic', 'worthiness'], dtype='object')

In [20]:
x_txt = np.stack(df['fasttext_embed'].map(unpck_txt), axis=0)
x_img = np.stack(df['image_embed'].map(unpck_img), axis=0)

In [22]:
y_tpc = to_categorical(np.array(df['topic']), N_TOPICS)
y_wts = to_categorical(np.array(df['worthiness']), N_WORTHINESSES)

In [23]:
topics_pickle_out = open("unpacked_worth_topics.pickle", "wb")
pickle.dump((x_img, x_txt, y_tpc, y_wts), topics_pickle_out)
topics_pickle_out.close()

### Preparing data

In [28]:
x_img_train, x_img_test, x_txt_train, x_txt_test, y_tpc_train, y_tpc_test, y_wts_train, y_wts_test = train_test_split(
    x_img, 
    x_txt, 
    y_tpc,
    y_wts,
    test_size=0.2, 
    random_state=42,
    stratify=y_tpc
)

x_img_train, x_img_val, x_txt_train, x_txt_val, y_tpc_train, y_tpc_val, y_wts_train, y_wts_val = train_test_split(
    x_img_train,
    x_txt_train,
    y_tpc_train,
    y_wts_train,
    test_size=0.2,
    random_state=42,
    stratify=y_tpc_train
)

In [29]:
img_sscaler = StandardScaler()
img_sscaler.fit(x_img_train)

x_img_train = img_sscaler.transform(x_img_train)
x_img_val = img_sscaler.transform(x_img_val)
x_img_test = img_sscaler.transform(x_img_test)

txt_sscaler = StandardScaler()
txt_sscaler.fit(x_txt_train)

x_txt_train = txt_sscaler.transform(x_txt_train)
x_txt_val = txt_sscaler.transform(x_txt_val)
x_txt_test = txt_sscaler.transform(x_txt_test)

In [33]:
x_img_train_t = torch.tensor(x_img_train).float()
x_img_val_t = torch.tensor(x_img_val).float()
x_img_test_t = torch.tensor(x_img_test).float()

x_txt_train_t = torch.tensor(x_txt_train).float()
x_txt_val_t = torch.tensor(x_txt_val).float()
x_txt_test_t = torch.tensor(x_txt_test).float()

y_tpc_train_t = torch.tensor(y_tpc_train).float()
y_tpc_val_t = torch.tensor(y_tpc_val).float()
y_tpc_test_t = torch.tensor(y_tpc_test).float()

y_wts_train_t = torch.tensor(y_wts_train).float()
y_wts_val_t = torch.tensor(y_wts_val).float()
y_wts_test_t = torch.tensor(y_wts_test).float()

In [35]:
train_ds = TensorDataset(x_img_train_t, x_txt_train_t, y_tpc_train_t, y_wts_train_t)
val_ds = TensorDataset(x_img_val_t, x_txt_val_t, y_tpc_val_t, y_wts_val_t)
test_ds = TensorDataset(x_img_test_t, x_txt_test_t, y_tpc_test_t, y_wts_test_t)

In [36]:
BATCH_SIZE = 2048

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [48]:
wts_train_ds = TensorDataset(x_img_train_t, x_txt_train_t, y_wts_train_t)
wts_val_ds = TensorDataset(x_img_val_t, x_txt_val_t, y_wts_val_t)
wts_test_ds = TensorDataset(x_img_test_t, x_txt_test_t, y_wts_test_t)

wts_train_loader = DataLoader(wts_train_ds, batch_size=BATCH_SIZE)
wts_val_loader = DataLoader(wts_val_ds, batch_size=BATCH_SIZE)
wts_test_loader = DataLoader(wts_test_ds, batch_size=BATCH_SIZE)

## Models

### worthiness only models

In [46]:
class WtsTrivialModel(nn.Module):
    def __init__(self, d=128, drop=0.25):
        super().__init__()

        self.fc_img = nn.Linear(IMG_LEN, d)
        self.fc_txt = nn.Linear(TXT_LEN, d)

        self.fc = nn.Linear(d * 2, d)
        self.out = nn.Linear(d, N_WORTHINESSES)

        self.dropout = nn.modules.Dropout(p=drop)
        
    def forward(self, inp_img, inp_txt):
        x_img = self.dropout(F.relu(self.fc_img(inp_img)))
        x_txt = self.dropout(F.relu(self.fc_txt(inp_txt)))

        x = torch.cat((x_img, x_txt), 1)
        x = self.dropout(F.relu(self.fc(x)))
        x = F.log_softmax(self.out(x), dim=1)
        return x

In [53]:
class WtsNormModelBN(nn.Module):
    def __init__(self, d=128, drop=0.25):
        super().__init__()

        self.fc_img_1 = nn.Linear(IMG_LEN, d * 4)
        self.bn_img_1 = nn.BatchNorm1d(num_features=d*4)
        self.fc_img_2 = nn.Linear(d * 4, d * 2)
        self.bn_img_2 = nn.BatchNorm1d(num_features=d*2)

        self.fc_txt_1 = nn.Linear(TXT_LEN, d * 2)
        self.bn_txt_1 = nn.BatchNorm1d(num_features=d*2)
        self.fc_txt_2 = nn.Linear(d * 2, d * 2)
        self.bn_txt_2 = nn.BatchNorm1d(num_features=d*2)

        self.fc1 = nn.Linear(d * 4, d)
        self.bn1 = nn.BatchNorm1d(num_features=d)
        self.fc2 = nn.Linear(d, d)
        self.bn2 = nn.BatchNorm1d(num_features=d)
        self.out = nn.Linear(d, N_WORTHINESSES)

        self.dropout = nn.modules.Dropout(p=drop)
        
    def forward(self, inp_img, inp_txt):
        x_img = self.bn_img_1(F.relu(self.fc_img_1(inp_img)))
        x_img = self.dropout(x_img)
        x_img = self.bn_img_2(F.relu(self.fc_img_2(x_img)))
        x_img = self.dropout(x_img)

        x_txt = self.bn_txt_1(F.relu(self.fc_txt_1(inp_txt)))
        x_txt = self.dropout(x_txt)
        x_txt = self.bn_txt_2(F.relu(self.fc_txt_2(x_txt)))
        x_txt = self.dropout(x_txt)

        x = torch.cat((x_img, x_txt), 1)
        x = self.bn1(F.relu(self.fc1(x)))
        x = self.dropout(x)
        x = self.bn2(F.relu(self.fc2(x)))

        x = F.log_softmax(self.out(x), dim=1)
        return x


In [60]:
class WtsTridentModelBN(nn.Module):
    def __init__(self, d=128, drop=0.25):
        super().__init__()

        self.fc_img_1 = nn.Linear(IMG_LEN, d * 4)
        self.bn_img_1 = nn.BatchNorm1d(num_features=d*4)
        self.fc_img_2 = nn.Linear(d * 4, d * 2)
        self.bn_img_2 = nn.BatchNorm1d(num_features=d*2)

        self.fc_txt_1 = nn.Linear(TXT_LEN, d * 2)
        self.bn_txt_1 = nn.BatchNorm1d(num_features=d*2)
        self.fc_txt_2 = nn.Linear(d * 2, d * 2)
        self.bn_txt_2 = nn.BatchNorm1d(num_features=d*2)

        self.fc1 = nn.Linear(d * 4, d)
        self.bn1 = nn.BatchNorm1d(num_features=d)
        self.fc2 = nn.Linear(d, d)
        self.bn2 = nn.BatchNorm1d(num_features=d)
        
        self.out = nn.Linear(d, N_WORTHINESSES)
        self.out_img = nn.Linear(d * 2, N_WORTHINESSES)
        self.out_txt = nn.Linear(d * 2, N_WORTHINESSES)
        
        self.dropout = nn.modules.Dropout(p=drop)
        
    def forward(self, inp_img, inp_txt):
        x_img = self.dropout(self.bn_img_1(F.relu(self.fc_img_1(inp_img))))
        x_img = self.dropout(self.bn_img_2(F.relu(self.fc_img_2(x_img))))

        x_txt = self.dropout(self.bn_txt_1(F.relu(self.fc_txt_1(inp_txt))))
        x_txt = self.dropout(self.bn_txt_2(F.relu(self.fc_txt_2(x_txt))))

        x = torch.cat((x_img, x_txt), 1)
        
        x = self.dropout(self.bn1(F.relu(self.fc1(x))))
        x = self.bn2(F.relu(self.fc2(x)))
        
        out = F.log_softmax(self.out(x), dim=1)
        out_img = F.log_softmax(self.out_img(x_img), dim=1)
        out_txt = F.log_softmax(self.out_txt(x_txt), dim=1)
        
        return out, out_img, out_txt

### function for fitting models

In [95]:
def fit_wts_model(
    model, 
    optimizer, 
    train_loader, 
    val_loader=None, 
    scheduler=None, 
    writer=None, 
    epochs=1, 
    es_dif=None, 
    es_tol=0,
    weight=None
):
    prev_train_loss = None
    tol_epochs = 0
    
    for epoch in range(epochs):
        model.train()

        train_loss_sum = 0.0
        train_loss_count = 0

        for x_img_cur, x_txt_cur, y_wts_cur in train_loader:
            model.zero_grad()
            output = model(x_img_cur, x_txt_cur)
                        
            train_loss = F.nll_loss(output, torch.argmax(y_wts_cur, dim=1), weight=weight)
            train_loss.backward()

            train_loss_sum += train_loss
            train_loss_count += 1

            optimizer.step()
            if scheduler is not None:
                scheduler.step()

        avg_train_loss = train_loss_sum / train_loss_count
        
        print('epoch:', epoch, 'train_loss:', train_loss, 'average train loss', avg_train_loss)
        if writer is not None:
            writer.add_scalar('train_loss', train_loss, epoch)
            writer.add_scalar('avg_train_loss', avg_train_loss, epoch)

        if val_loader is not None:
            model.eval()

            correct = 0
            total = 0
            val_loss_sum = 0.0
            val_loss_count = 0

            with torch.no_grad():
                for x_img_cur, x_txt_cur, y_wts_cur in val_loader:
                    output = model(x_img_cur, x_txt_cur)
                    val_loss = F.nll_loss(output, torch.argmax(y_wts_cur, dim=1))
                    val_loss_sum += val_loss
                    val_loss_count += 1
                    for idx, i in enumerate(output):
                        if torch.argmax(i) == torch.argmax(y_wts_cur, dim=1)[idx]:
                            correct += 1
                        total += 1

            print('val_acc:', correct / total, 'val_avg_loss:', val_loss_sum / val_loss_count)
            if writer is not None:
                writer.add_scalar('val_acc', correct / total, epoch)
                writer.add_scalar('val_avg_loss', val_loss_sum / val_loss_count, epoch)
                
        # es part
        if es_dif is not None and epoch != 0:
            if tol_epochs != 0:  # already in tolerance mode
                if prev_train_loss - avg_train_loss > es_dif: # leave tolerance mode
                    tol_epochs = 0
                elif tol_epochs >= es_tol: # tolerance limit exceeded
                    return
                else: # continue tolerance mode
                    tol_epochs += 1
            elif prev_train_loss - avg_train_loss <= es_dif: # not in tolerance but to slow learning
                if es_tol == 0:  # no tolerance
                    return 
                else: #enter tolerance mode
                    tol_epochs += 1
        prev_train_loss = avg_train_loss

In [125]:
def fit_wts_trident_model(
    model, 
    optimizer, 
    train_loader, 
    val_loader=None, 
    scheduler=None, 
    writer=None, 
    epochs=1, 
    es_dif=None, 
    es_tol=0,
    weight=None
):
    prev_train_loss = None
    tol_epochs = 0
    
    for epoch in range(epochs):
        model.train()

        train_loss_sum = 0.0
        train_loss_common_sum = 0.0
        train_loss_img_sum = 0.0
        train_loss_txt_sum = 0.0
        train_loss_count = 0

        for x_img_cur, x_txt_cur, y_wts_cur in train_loader:
            model.zero_grad()
            out_common, out_img, out_txt = model(x_img_cur, x_txt_cur)
                        
            train_loss_common = F.nll_loss(out_common, torch.argmax(y_wts_cur, dim=1), weight=weight)
            train_loss_img = F.nll_loss(out_img, torch.argmax(y_wts_cur, dim=1), weight=weight)
            train_loss_txt = F.nll_loss(out_txt, torch.argmax(y_wts_cur, dim=1), weight=weight)
            train_loss = train_loss_common + train_loss_img + train_loss_txt
            train_loss.backward()

            train_loss_sum += train_loss
            train_loss_common_sum += train_loss_common
            train_loss_img_sum += train_loss_img
            train_loss_txt_sum += train_loss_txt
            train_loss_count += 1

            optimizer.step()
            if scheduler is not None:
                scheduler.step()

        avg_train_loss = train_loss_sum / train_loss_count
        
        print('epoch:', epoch, 'train_loss:', train_loss, 'average train loss', avg_train_loss)
        if writer is not None:
            writer.add_scalar('train_loss', train_loss, epoch)
            writer.add_scalar('avg_train_loss', train_loss_sum / train_loss_count, epoch)
            writer.add_scalar('avg_train_loss_common', train_loss_common_sum / train_loss_count, epoch)
            writer.add_scalar('avg_train_loss_img', train_loss_img_sum / train_loss_count, epoch)
            writer.add_scalar('avg_train_loss_txt', train_loss_txt_sum / train_loss_count, epoch)


        if val_loader is not None:
            model.eval()

            correct_common = 0
            correct_img = 0
            correct_txt = 0
            total = 0
            
            val_loss_common_sum = 0.0
            val_loss_img_sum = 0.0
            val_loss_txt_sum = 0.0
            val_loss_sum = 0.0
            val_loss_count = 0

            with torch.no_grad():
                for x_img_cur, x_txt_cur, y_wts_cur in val_loader:
                    out_common, out_img, out_txt = model(x_img_cur, x_txt_cur)
                    target = torch.argmax(y_wts_cur, dim=1)
                    val_loss_common = F.nll_loss(out_common, target)
                    val_loss_img = F.nll_loss(out_img, target)
                    val_loss_txt = F.nll_loss(out_txt, target)
                    
                    val_loss = val_loss_common + val_loss_img + val_loss_txt
                    
                    val_loss_common_sum += val_loss_common
                    val_loss_img_sum += val_loss_img
                    val_loss_txt_sum += val_loss_txt
                    val_loss_sum += val_loss
                    
                    val_loss_count += 1
                    for idx, i in enumerate(out_common):
                        if torch.argmax(i) == target[idx]:
                            correct_common += 1
                        total += 1
                    
                    for idx, i in enumerate(out_img):
                        if torch.argmax(i) == target[idx]:
                            correct_img += 1
                           
                    for idx, i in enumerate(out_txt):
                        if torch.argmax(i) == target[idx]:
                            correct_txt += 1
                            
            print('val_acc:', correct_common / total, 'val_avg_loss:', val_loss_sum / val_loss_count)
            if writer is not None:
                writer.add_scalar('val_acc', correct_common / total, epoch)
                writer.add_scalar('val_img_acc', correct_img / total, epoch)
                writer.add_scalar('val_txt_acc', correct_txt / total, epoch)
                writer.add_scalar('val_avg_loss', val_loss_sum / val_loss_count, epoch)
                writer.add_scalar('val_avg_common_loss', val_loss_common_sum / val_loss_count, epoch)
                writer.add_scalar('val_avg_img_loss', val_loss_img_sum / val_loss_count, epoch)
                writer.add_scalar('val_avg_txt_loss', val_loss_txt_sum / val_loss_count, epoch)

                

        # es part
        if es_dif is not None and epoch != 0:
            if tol_epochs != 0:  # already in tolerance mode
                if prev_train_loss - avg_train_loss > es_dif: # leave tolerance mode
                    tol_epochs = 0
                elif tol_epochs >= es_tol: # tolerance limit exceeded
                    return
                else: # continue tolerance mode
                    tol_epochs += 1
            elif prev_train_loss - avg_train_loss <= es_dif: # not in tolerance but to slow learning
                if es_tol == 0:  # no tolerance
                    return 
                else: #enter tolerance mode
                    tol_epochs += 1
        prev_train_loss = avg_train_loss

## Experiments

In [92]:
model = WtsTrivialModel(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
writer = SummaryWriter('runs/trivial_d128_drop05_wd0005_weighted')

fit_wts_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    writer=writer,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader
)

epoch: 0 train_loss: tensor(0.6910, grad_fn=<NllLossBackward>) average train loss tensor(0.6613, grad_fn=<DivBackward0>)
val_acc: 0.644426746895555 val_avg_loss: tensor(0.6232)
epoch: 1 train_loss: tensor(0.6621, grad_fn=<NllLossBackward>) average train loss tensor(0.6268, grad_fn=<DivBackward0>)
val_acc: 0.6293543344246821 val_avg_loss: tensor(0.6179)
epoch: 2 train_loss: tensor(0.5958, grad_fn=<NllLossBackward>) average train loss tensor(0.6163, grad_fn=<DivBackward0>)
val_acc: 0.6619768162109548 val_avg_loss: tensor(0.5890)
epoch: 3 train_loss: tensor(0.5158, grad_fn=<NllLossBackward>) average train loss tensor(0.6028, grad_fn=<DivBackward0>)
val_acc: 0.6561366250774268 val_avg_loss: tensor(0.5834)
epoch: 4 train_loss: tensor(0.5090, grad_fn=<NllLossBackward>) average train loss tensor(0.5984, grad_fn=<DivBackward0>)
val_acc: 0.6498834911364776 val_avg_loss: tensor(0.5862)
epoch: 5 train_loss: tensor(0.4845, grad_fn=<NllLossBackward>) average train loss tensor(0.5859, grad_fn=<DivBa

epoch: 46 train_loss: tensor(0.1785, grad_fn=<NllLossBackward>) average train loss tensor(0.3617, grad_fn=<DivBackward0>)
val_acc: 0.8509276465209569 val_avg_loss: tensor(0.3151)
epoch: 47 train_loss: tensor(0.2254, grad_fn=<NllLossBackward>) average train loss tensor(0.3536, grad_fn=<DivBackward0>)
val_acc: 0.8284517594313188 val_avg_loss: tensor(0.3462)
epoch: 48 train_loss: tensor(0.1739, grad_fn=<NllLossBackward>) average train loss tensor(0.3477, grad_fn=<DivBackward0>)
val_acc: 0.8443795534318497 val_avg_loss: tensor(0.3276)
epoch: 49 train_loss: tensor(0.1817, grad_fn=<NllLossBackward>) average train loss tensor(0.3413, grad_fn=<DivBackward0>)
val_acc: 0.8369760788130843 val_avg_loss: tensor(0.3356)
epoch: 50 train_loss: tensor(0.1930, grad_fn=<NllLossBackward>) average train loss tensor(0.3381, grad_fn=<DivBackward0>)
val_acc: 0.8482140223579034 val_avg_loss: tensor(0.3174)
epoch: 51 train_loss: tensor(0.1936, grad_fn=<NllLossBackward>) average train loss tensor(0.3384, grad_fn

epoch: 92 train_loss: tensor(0.1372, grad_fn=<NllLossBackward>) average train loss tensor(0.2653, grad_fn=<DivBackward0>)
val_acc: 0.8933132761112585 val_avg_loss: tensor(0.2675)
epoch: 93 train_loss: tensor(0.1349, grad_fn=<NllLossBackward>) average train loss tensor(0.2663, grad_fn=<DivBackward0>)
val_acc: 0.890924107011179 val_avg_loss: tensor(0.2682)
epoch: 94 train_loss: tensor(0.1110, grad_fn=<NllLossBackward>) average train loss tensor(0.2632, grad_fn=<DivBackward0>)
val_acc: 0.8967348022298912 val_avg_loss: tensor(0.2607)
epoch: 95 train_loss: tensor(0.1075, grad_fn=<NllLossBackward>) average train loss tensor(0.2564, grad_fn=<DivBackward0>)
val_acc: 0.9025749933634192 val_avg_loss: tensor(0.2553)
epoch: 96 train_loss: tensor(0.1088, grad_fn=<NllLossBackward>) average train loss tensor(0.2566, grad_fn=<DivBackward0>)
val_acc: 0.8943751290446273 val_avg_loss: tensor(0.2672)
epoch: 97 train_loss: tensor(0.1374, grad_fn=<NllLossBackward>) average train loss tensor(0.2607, grad_fn=

In [93]:
predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))

              precision    recall  f1-score   support

           0       0.98      0.91      0.94     41251
           1       0.06      0.21      0.10      1128

    accuracy                           0.89     42379
   macro avg       0.52      0.56      0.52     42379
weighted avg       0.95      0.89      0.92     42379



In [103]:
weights_list = [torch.tensor([1, 1000]).float(), 
                torch.tensor([1, 500]).float(), 
                torch.tensor([1, 400]).float(),
                torch.tensor([1, 300]).float(),
                torch.tensor([1, 200]).float(),
                torch.tensor([1, 100]).float()]


for weight in weights_list:
    print('train with weight', weight)
    
    model = WtsTrivialModel(d=128, drop=0.5)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
    writer = SummaryWriter('runs/trivial_d128_drop05_wd0005_weighted' + str(int(weight[1].item())))

    fit_wts_model(
        model=model,
        optimizer=optimizer,
        epochs=100,
        writer=writer,
        train_loader=wts_train_loader,
        val_loader=wts_val_loader,
        weight=weight
    )
    
    predictions = torch.tensor([])

    with torch.no_grad():
        for x_img_cur, x_txt_cur, _ in wts_test_loader:
            outputs = model(x_img_cur.float(), x_txt_cur.float())
            predictions = torch.cat((predictions, outputs), 0)

    predictions = predictions.numpy()

    print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))

train with weight tensor([   1., 1000.])
epoch: 0 train_loss: tensor(0.2138, grad_fn=<NllLossBackward>) average train loss tensor(0.1830, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(2.9024)
epoch: 1 train_loss: tensor(0.1734, grad_fn=<NllLossBackward>) average train loss tensor(0.1501, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(2.7815)
epoch: 2 train_loss: tensor(0.1702, grad_fn=<NllLossBackward>) average train loss tensor(0.1467, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(2.7375)
epoch: 3 train_loss: tensor(0.1773, grad_fn=<NllLossBackward>) average train loss tensor(0.1442, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(2.7738)
epoch: 4 train_loss: tensor(0.1639, grad_fn=<NllLossBackward>) average train loss tensor(0.1419, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(2.7092)
epoch: 5 train_loss: tensor(0.1610, grad_fn=<NllLossBackwar

val_acc: 0.12960504970061645 val_avg_loss: tensor(2.6513)
epoch: 46 train_loss: tensor(0.1428, grad_fn=<NllLossBackward>) average train loss tensor(0.1274, grad_fn=<DivBackward0>)
val_acc: 0.12966404153024805 val_avg_loss: tensor(2.5490)
epoch: 47 train_loss: tensor(0.1420, grad_fn=<NllLossBackward>) average train loss tensor(0.1288, grad_fn=<DivBackward0>)
val_acc: 0.1468601598678583 val_avg_loss: tensor(2.4590)
epoch: 48 train_loss: tensor(0.1369, grad_fn=<NllLossBackward>) average train loss tensor(0.1270, grad_fn=<DivBackward0>)
val_acc: 0.13866029554906645 val_avg_loss: tensor(2.4702)
epoch: 49 train_loss: tensor(0.1427, grad_fn=<NllLossBackward>) average train loss tensor(0.1258, grad_fn=<DivBackward0>)
val_acc: 0.1468896557826741 val_avg_loss: tensor(2.4384)
epoch: 50 train_loss: tensor(0.1455, grad_fn=<NllLossBackward>) average train loss tensor(0.1304, grad_fn=<DivBackward0>)
val_acc: 0.10583134235908327 val_avg_loss: tensor(2.5593)
epoch: 51 train_loss: tensor(0.1435, grad_fn

val_acc: 0.19116302392118692 val_avg_loss: tensor(2.3821)
epoch: 92 train_loss: tensor(0.1286, grad_fn=<NllLossBackward>) average train loss tensor(0.1228, grad_fn=<DivBackward0>)
val_acc: 0.18440845942836917 val_avg_loss: tensor(2.3511)
epoch: 93 train_loss: tensor(0.1392, grad_fn=<NllLossBackward>) average train loss tensor(0.1233, grad_fn=<DivBackward0>)
val_acc: 0.18883284665073888 val_avg_loss: tensor(2.4771)
epoch: 94 train_loss: tensor(0.1361, grad_fn=<NllLossBackward>) average train loss tensor(0.1229, grad_fn=<DivBackward0>)
val_acc: 0.16352535173878419 val_avg_loss: tensor(2.4365)
epoch: 95 train_loss: tensor(0.1500, grad_fn=<NllLossBackward>) average train loss tensor(0.1247, grad_fn=<DivBackward0>)
val_acc: 0.13264312892664365 val_avg_loss: tensor(2.6246)
epoch: 96 train_loss: tensor(0.1281, grad_fn=<NllLossBackward>) average train loss tensor(0.1229, grad_fn=<DivBackward0>)
val_acc: 0.1944665663805563 val_avg_loss: tensor(2.2681)
epoch: 97 train_loss: tensor(0.1324, grad_f

val_acc: 0.17293454856502374 val_avg_loss: tensor(1.8627)
epoch: 36 train_loss: tensor(0.1768, grad_fn=<NllLossBackward>) average train loss tensor(0.1902, grad_fn=<DivBackward0>)
val_acc: 0.23036309471138247 val_avg_loss: tensor(1.7941)
epoch: 37 train_loss: tensor(0.1863, grad_fn=<NllLossBackward>) average train loss tensor(0.1883, grad_fn=<DivBackward0>)
val_acc: 0.24425567058962333 val_avg_loss: tensor(1.7728)
epoch: 38 train_loss: tensor(0.1892, grad_fn=<NllLossBackward>) average train loss tensor(0.1906, grad_fn=<DivBackward0>)
val_acc: 0.2328112556410937 val_avg_loss: tensor(1.8272)
epoch: 39 train_loss: tensor(0.1965, grad_fn=<NllLossBackward>) average train loss tensor(0.1934, grad_fn=<DivBackward0>)
val_acc: 0.23697017963012124 val_avg_loss: tensor(1.8516)
epoch: 40 train_loss: tensor(0.1978, grad_fn=<NllLossBackward>) average train loss tensor(0.1905, grad_fn=<DivBackward0>)
val_acc: 0.22753148688906585 val_avg_loss: tensor(1.9036)
epoch: 41 train_loss: tensor(0.2004, grad_f

val_acc: 0.29554906645429607 val_avg_loss: tensor(1.7430)
epoch: 82 train_loss: tensor(0.1655, grad_fn=<NllLossBackward>) average train loss tensor(0.1826, grad_fn=<DivBackward0>)
val_acc: 0.28183346606495 val_avg_loss: tensor(1.6410)
epoch: 83 train_loss: tensor(0.1882, grad_fn=<NllLossBackward>) average train loss tensor(0.1769, grad_fn=<DivBackward0>)
val_acc: 0.29150812612453175 val_avg_loss: tensor(1.7308)
epoch: 84 train_loss: tensor(0.1712, grad_fn=<NllLossBackward>) average train loss tensor(0.1786, grad_fn=<DivBackward0>)
val_acc: 0.302716573754535 val_avg_loss: tensor(1.7116)
epoch: 85 train_loss: tensor(0.1696, grad_fn=<NllLossBackward>) average train loss tensor(0.1834, grad_fn=<DivBackward0>)
val_acc: 0.29183258118750555 val_avg_loss: tensor(1.6295)
epoch: 86 train_loss: tensor(0.1663, grad_fn=<NllLossBackward>) average train loss tensor(0.1755, grad_fn=<DivBackward0>)
val_acc: 0.3292334011739374 val_avg_loss: tensor(1.5960)
epoch: 87 train_loss: tensor(0.1689, grad_fn=<Nl

...

val_acc: 0.14635872931598973 val_avg_loss: tensor(1.4520)
epoch: 6 train_loss: tensor(0.3679, grad_fn=<NllLossBackward>) average train loss tensor(0.3698, grad_fn=<DivBackward0>)
val_acc: 0.1720496711205498 val_avg_loss: tensor(1.4111)
epoch: 7 train_loss: tensor(0.3580, grad_fn=<NllLossBackward>) average train loss tensor(0.3661, grad_fn=<DivBackward0>)
val_acc: 0.17921717842078871 val_avg_loss: tensor(1.3539)
epoch: 8 train_loss: tensor(0.3449, grad_fn=<NllLossBackward>) average train loss tensor(0.3603, grad_fn=<DivBackward0>)
val_acc: 0.23776656933014778 val_avg_loss: tensor(1.3558)
epoch: 9 train_loss: tensor(0.3505, grad_fn=<NllLossBackward>) average train loss tensor(0.3589, grad_fn=<DivBackward0>)
val_acc: 0.22614517889272337 val_avg_loss: tensor(1.3700)
epoch: 10 train_loss: tensor(0.3499, grad_fn=<NllLossBackward>) average train loss tensor(0.3585, grad_fn=<DivBackward0>)
val_acc: 0.22487685455564405 val_avg_loss: tensor(1.3432)
epoch: 11 train_loss: tensor(0.3273, grad_fn=<N

val_acc: 0.48724301684216736 val_avg_loss: tensor(0.9537)
epoch: 52 train_loss: tensor(0.2437, grad_fn=<NllLossBackward>) average train loss tensor(0.2781, grad_fn=<DivBackward0>)
val_acc: 0.4645901542636345 val_avg_loss: tensor(0.9724)
epoch: 53 train_loss: tensor(0.2875, grad_fn=<NllLossBackward>) average train loss tensor(0.2729, grad_fn=<DivBackward0>)
val_acc: 0.44299914461847034 val_avg_loss: tensor(1.0051)
epoch: 54 train_loss: tensor(0.2236, grad_fn=<NllLossBackward>) average train loss tensor(0.2760, grad_fn=<DivBackward0>)
val_acc: 0.47818777099371734 val_avg_loss: tensor(0.9127)
epoch: 55 train_loss: tensor(0.2227, grad_fn=<NllLossBackward>) average train loss tensor(0.2713, grad_fn=<DivBackward0>)
val_acc: 0.47709642214553283 val_avg_loss: tensor(0.9509)
epoch: 56 train_loss: tensor(0.2072, grad_fn=<NllLossBackward>) average train loss tensor(0.2662, grad_fn=<DivBackward0>)
val_acc: 0.5021089579093295 val_avg_loss: tensor(0.8981)
epoch: 57 train_loss: tensor(0.2235, grad_fn

val_acc: 0.5441701324366576 val_avg_loss: tensor(0.8738)
epoch: 98 train_loss: tensor(0.1962, grad_fn=<NllLossBackward>) average train loss tensor(0.2380, grad_fn=<DivBackward0>)
val_acc: 0.5528124354776863 val_avg_loss: tensor(0.8742)
epoch: 99 train_loss: tensor(0.1968, grad_fn=<NllLossBackward>) average train loss tensor(0.2411, grad_fn=<DivBackward0>)
val_acc: 0.5534908415184497 val_avg_loss: tensor(0.8810)
              precision    recall  f1-score   support

           0       0.99      0.55      0.71     41251
           1       0.04      0.70      0.08      1128

    accuracy                           0.56     42379
   macro avg       0.51      0.63      0.39     42379
weighted avg       0.96      0.56      0.69     42379

train with weight tensor([  1., 100.])
epoch: 0 train_loss: tensor(0.6013, grad_fn=<NllLossBackward>) average train loss tensor(0.5626, grad_fn=<DivBackward0>)
val_acc: 0.17697548889478806 val_avg_loss: tensor(1.0359)
epoch: 1 train_loss: tensor(0.5615, grad

val_acc: 0.6008022888829897 val_avg_loss: tensor(0.6548)
epoch: 42 train_loss: tensor(0.2510, grad_fn=<NllLossBackward>) average train loss tensor(0.3455, grad_fn=<DivBackward0>)
val_acc: 0.6241335575022859 val_avg_loss: tensor(0.5999)
epoch: 43 train_loss: tensor(0.2410, grad_fn=<NllLossBackward>) average train loss tensor(0.3439, grad_fn=<DivBackward0>)
val_acc: 0.6059050821461227 val_avg_loss: tensor(0.6338)
epoch: 44 train_loss: tensor(0.2822, grad_fn=<NllLossBackward>) average train loss tensor(0.3429, grad_fn=<DivBackward0>)
val_acc: 0.6605315163849806 val_avg_loss: tensor(0.5740)
epoch: 45 train_loss: tensor(0.1854, grad_fn=<NllLossBackward>) average train loss tensor(0.3401, grad_fn=<DivBackward0>)
val_acc: 0.6675220481963248 val_avg_loss: tensor(0.5591)
epoch: 46 train_loss: tensor(0.2205, grad_fn=<NllLossBackward>) average train loss tensor(0.3354, grad_fn=<DivBackward0>)
val_acc: 0.6570804943515323 val_avg_loss: tensor(0.5602)
epoch: 47 train_loss: tensor(0.2093, grad_fn=<Nl

val_acc: 0.7319706220688434 val_avg_loss: tensor(0.5057)
epoch: 88 train_loss: tensor(0.1831, grad_fn=<NllLossBackward>) average train loss tensor(0.2826, grad_fn=<DivBackward0>)
val_acc: 0.7511429666991122 val_avg_loss: tensor(0.4701)
epoch: 89 train_loss: tensor(0.1615, grad_fn=<NllLossBackward>) average train loss tensor(0.2690, grad_fn=<DivBackward0>)
val_acc: 0.7592543432734566 val_avg_loss: tensor(0.4531)
epoch: 90 train_loss: tensor(0.1635, grad_fn=<NllLossBackward>) average train loss tensor(0.2655, grad_fn=<DivBackward0>)
val_acc: 0.7813762793853052 val_avg_loss: tensor(0.4267)
epoch: 91 train_loss: tensor(0.1766, grad_fn=<NllLossBackward>) average train loss tensor(0.2821, grad_fn=<DivBackward0>)
val_acc: 0.7385187151579506 val_avg_loss: tensor(0.4789)
epoch: 92 train_loss: tensor(0.1904, grad_fn=<NllLossBackward>) average train loss tensor(0.2634, grad_fn=<DivBackward0>)
val_acc: 0.7601097248031148 val_avg_loss: tensor(0.4667)
epoch: 93 train_loss: tensor(0.1721, grad_fn=<Nl

ROC AUC Score with weights (1, 100)

In [105]:
roc_auc_score(y_wts_test, predictions)

0.664015070728567

In [107]:
model = WtsTrivialModel(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)

fit_wts_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader,
    weight=torch.tensor([1, 200]).float()
)
    
predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))
print('roc auc score:', roc_auc_score(y_wts_test, predictions))

epoch: 0 train_loss: tensor(0.5179, grad_fn=<NllLossBackward>) average train loss tensor(0.4334, grad_fn=<DivBackward0>)
val_acc: 0.025336990826770493 val_avg_loss: tensor(1.5276)
epoch: 1 train_loss: tensor(0.4391, grad_fn=<NllLossBackward>) average train loss tensor(0.4035, grad_fn=<DivBackward0>)
val_acc: 0.06497950033920302 val_avg_loss: tensor(1.4921)
epoch: 2 train_loss: tensor(0.4528, grad_fn=<NllLossBackward>) average train loss tensor(0.3961, grad_fn=<DivBackward0>)
val_acc: 0.07486063180249536 val_avg_loss: tensor(1.4577)
epoch: 3 train_loss: tensor(0.3817, grad_fn=<NllLossBackward>) average train loss tensor(0.3879, grad_fn=<DivBackward0>)
val_acc: 0.10630327699613604 val_avg_loss: tensor(1.4352)
epoch: 4 train_loss: tensor(0.3902, grad_fn=<NllLossBackward>) average train loss tensor(0.3842, grad_fn=<DivBackward0>)
val_acc: 0.12473822375600979 val_avg_loss: tensor(1.4462)
epoch: 5 train_loss: tensor(0.3996, grad_fn=<NllLossBackward>) average train loss tensor(0.3790, grad_fn

epoch: 46 train_loss: tensor(0.2364, grad_fn=<NllLossBackward>) average train loss tensor(0.2874, grad_fn=<DivBackward0>)
val_acc: 0.442025779429549 val_avg_loss: tensor(0.9872)
epoch: 47 train_loss: tensor(0.2486, grad_fn=<NllLossBackward>) average train loss tensor(0.2880, grad_fn=<DivBackward0>)
val_acc: 0.4347402884700469 val_avg_loss: tensor(1.0183)
epoch: 48 train_loss: tensor(0.2358, grad_fn=<NllLossBackward>) average train loss tensor(0.2749, grad_fn=<DivBackward0>)
val_acc: 0.45928088959679086 val_avg_loss: tensor(1.0019)
epoch: 49 train_loss: tensor(0.2290, grad_fn=<NllLossBackward>) average train loss tensor(0.2846, grad_fn=<DivBackward0>)
val_acc: 0.4297259829513612 val_avg_loss: tensor(1.0017)
epoch: 50 train_loss: tensor(0.2281, grad_fn=<NllLossBackward>) average train loss tensor(0.2786, grad_fn=<DivBackward0>)
val_acc: 0.4459782320148659 val_avg_loss: tensor(0.9995)
epoch: 51 train_loss: tensor(0.2469, grad_fn=<NllLossBackward>) average train loss tensor(0.2775, grad_fn

epoch: 92 train_loss: tensor(0.1984, grad_fn=<NllLossBackward>) average train loss tensor(0.2394, grad_fn=<DivBackward0>)
val_acc: 0.5348789192696811 val_avg_loss: tensor(0.9184)
epoch: 93 train_loss: tensor(0.1619, grad_fn=<NllLossBackward>) average train loss tensor(0.2417, grad_fn=<DivBackward0>)
val_acc: 0.5772055570303513 val_avg_loss: tensor(0.7990)
epoch: 94 train_loss: tensor(0.1903, grad_fn=<NllLossBackward>) average train loss tensor(0.2420, grad_fn=<DivBackward0>)
val_acc: 0.5399522166179984 val_avg_loss: tensor(0.8480)
epoch: 95 train_loss: tensor(0.1933, grad_fn=<NllLossBackward>) average train loss tensor(0.2429, grad_fn=<DivBackward0>)
val_acc: 0.5180957437394921 val_avg_loss: tensor(0.8942)
epoch: 96 train_loss: tensor(0.1803, grad_fn=<NllLossBackward>) average train loss tensor(0.2414, grad_fn=<DivBackward0>)
val_acc: 0.5637849157891632 val_avg_loss: tensor(0.8178)
epoch: 97 train_loss: tensor(0.1576, grad_fn=<NllLossBackward>) average train loss tensor(0.2367, grad_fn

In [108]:
model = WtsTrivialModel(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)

fit_wts_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader,
    weight=torch.tensor([1, 300]).float()
)
    
predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))
print('roc auc score:', roc_auc_score(y_wts_test, predictions))

epoch: 0 train_loss: tensor(0.4383, grad_fn=<NllLossBackward>) average train loss tensor(0.3738, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(1.8072)
epoch: 1 train_loss: tensor(0.3928, grad_fn=<NllLossBackward>) average train loss tensor(0.3268, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(1.8049)
epoch: 2 train_loss: tensor(0.3436, grad_fn=<NllLossBackward>) average train loss tensor(0.3198, grad_fn=<DivBackward0>)
val_acc: 0.04279857239772292 val_avg_loss: tensor(1.7705)
epoch: 3 train_loss: tensor(0.3544, grad_fn=<NllLossBackward>) average train loss tensor(0.3151, grad_fn=<DivBackward0>)
val_acc: 0.07987493732118102 val_avg_loss: tensor(1.7618)
epoch: 4 train_loss: tensor(0.3098, grad_fn=<NllLossBackward>) average train loss tensor(0.3112, grad_fn=<DivBackward0>)
val_acc: 0.0755390378432587 val_avg_loss: tensor(1.7231)
epoch: 5 train_loss: tensor(0.3114, grad_fn=<NllLossBackward>) average train loss tensor(0.3056, grad_fn

val_acc: 0.28985635489484707 val_avg_loss: tensor(1.4446)
epoch: 46 train_loss: tensor(0.2377, grad_fn=<NllLossBackward>) average train loss tensor(0.2437, grad_fn=<DivBackward0>)
val_acc: 0.31062147892516884 val_avg_loss: tensor(1.4951)
epoch: 47 train_loss: tensor(0.2079, grad_fn=<NllLossBackward>) average train loss tensor(0.2439, grad_fn=<DivBackward0>)
val_acc: 0.3378167123853346 val_avg_loss: tensor(1.3291)
epoch: 48 train_loss: tensor(0.2355, grad_fn=<NllLossBackward>) average train loss tensor(0.2455, grad_fn=<DivBackward0>)
val_acc: 0.33607645341120257 val_avg_loss: tensor(1.3915)
epoch: 49 train_loss: tensor(0.2134, grad_fn=<NllLossBackward>) average train loss tensor(0.2426, grad_fn=<DivBackward0>)
val_acc: 0.35486535114886586 val_avg_loss: tensor(1.3737)
epoch: 50 train_loss: tensor(0.2200, grad_fn=<NllLossBackward>) average train loss tensor(0.2410, grad_fn=<DivBackward0>)
val_acc: 0.33067870099991153 val_avg_loss: tensor(1.3847)
epoch: 51 train_loss: tensor(0.2188, grad_f

val_acc: 0.397486948057694 val_avg_loss: tensor(1.2477)
epoch: 92 train_loss: tensor(0.1650, grad_fn=<NllLossBackward>) average train loss tensor(0.2142, grad_fn=<DivBackward0>)
val_acc: 0.4518774149780255 val_avg_loss: tensor(1.2052)
epoch: 93 train_loss: tensor(0.2082, grad_fn=<NllLossBackward>) average train loss tensor(0.2212, grad_fn=<DivBackward0>)
val_acc: 0.42668790372533405 val_avg_loss: tensor(1.2524)
epoch: 94 train_loss: tensor(0.1912, grad_fn=<NllLossBackward>) average train loss tensor(0.2170, grad_fn=<DivBackward0>)
val_acc: 0.4168952600064891 val_avg_loss: tensor(1.2893)
epoch: 95 train_loss: tensor(0.1957, grad_fn=<NllLossBackward>) average train loss tensor(0.2162, grad_fn=<DivBackward0>)
val_acc: 0.47340943279355807 val_avg_loss: tensor(1.0927)
epoch: 96 train_loss: tensor(0.1794, grad_fn=<NllLossBackward>) average train loss tensor(0.2183, grad_fn=<DivBackward0>)
val_acc: 0.4673627702563195 val_avg_loss: tensor(1.0801)
epoch: 97 train_loss: tensor(0.1864, grad_fn=<N

In [109]:
model = WtsTrivialModel(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)

fit_wts_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader,
    weight=torch.tensor([1, 400]).float()
)
    
predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))
print('roc auc score:', roc_auc_score(y_wts_test, predictions))

epoch: 0 train_loss: tensor(0.3634, grad_fn=<NllLossBackward>) average train loss tensor(0.3120, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(2.0965)
epoch: 1 train_loss: tensor(0.3425, grad_fn=<NllLossBackward>) average train loss tensor(0.2763, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(2.0036)
epoch: 2 train_loss: tensor(0.3149, grad_fn=<NllLossBackward>) average train loss tensor(0.2693, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(2.0176)
epoch: 3 train_loss: tensor(0.3172, grad_fn=<NllLossBackward>) average train loss tensor(0.2669, grad_fn=<DivBackward0>)
val_acc: 0.02657581924903401 val_avg_loss: tensor(2.0324)
epoch: 4 train_loss: tensor(0.2759, grad_fn=<NllLossBackward>) average train loss tensor(0.2626, grad_fn=<DivBackward0>)
val_acc: 0.05238474471285727 val_avg_loss: tensor(1.9943)
epoch: 5 train_loss: tensor(0.2817, grad_fn=<NllLossBackward>) average train loss tensor(0.2595, grad_

val_acc: 0.2687372798867357 val_avg_loss: tensor(1.6537)
epoch: 46 train_loss: tensor(0.2060, grad_fn=<NllLossBackward>) average train loss tensor(0.2080, grad_fn=<DivBackward0>)
val_acc: 0.2880276081762676 val_avg_loss: tensor(1.6281)
epoch: 47 train_loss: tensor(0.1985, grad_fn=<NllLossBackward>) average train loss tensor(0.2143, grad_fn=<DivBackward0>)
val_acc: 0.2794442969648704 val_avg_loss: tensor(1.6457)
epoch: 48 train_loss: tensor(0.2017, grad_fn=<NllLossBackward>) average train loss tensor(0.2125, grad_fn=<DivBackward0>)
val_acc: 0.2531339409491785 val_avg_loss: tensor(1.6799)
epoch: 49 train_loss: tensor(0.2062, grad_fn=<NllLossBackward>) average train loss tensor(0.2118, grad_fn=<DivBackward0>)
val_acc: 0.23838598354127954 val_avg_loss: tensor(1.6391)
epoch: 50 train_loss: tensor(0.1915, grad_fn=<NllLossBackward>) average train loss tensor(0.2071, grad_fn=<DivBackward0>)
val_acc: 0.28560894316137214 val_avg_loss: tensor(1.6339)
epoch: 51 train_loss: tensor(0.1921, grad_fn=<

val_acc: 0.3596436893490252 val_avg_loss: tensor(1.4422)
epoch: 92 train_loss: tensor(0.1874, grad_fn=<NllLossBackward>) average train loss tensor(0.1992, grad_fn=<DivBackward0>)
val_acc: 0.33713830634457126 val_avg_loss: tensor(1.4707)
epoch: 93 train_loss: tensor(0.1986, grad_fn=<NllLossBackward>) average train loss tensor(0.2054, grad_fn=<DivBackward0>)
val_acc: 0.28510751260950357 val_avg_loss: tensor(1.5390)
epoch: 94 train_loss: tensor(0.1808, grad_fn=<NllLossBackward>) average train loss tensor(0.1982, grad_fn=<DivBackward0>)
val_acc: 0.34610506444857386 val_avg_loss: tensor(1.4209)
epoch: 95 train_loss: tensor(0.1622, grad_fn=<NllLossBackward>) average train loss tensor(0.1994, grad_fn=<DivBackward0>)
val_acc: 0.3464590154263634 val_avg_loss: tensor(1.4567)
epoch: 96 train_loss: tensor(0.1893, grad_fn=<NllLossBackward>) average train loss tensor(0.1962, grad_fn=<DivBackward0>)
val_acc: 0.3500870129487066 val_avg_loss: tensor(1.4367)
epoch: 97 train_loss: tensor(0.1803, grad_fn=

In [110]:
model = WtsTrivialModel(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)

fit_wts_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader,
    weight=torch.tensor([1, 500]).float()
)
    
predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))
print('roc auc score:', roc_auc_score(y_wts_test, predictions))

epoch: 0 train_loss: tensor(0.3379, grad_fn=<NllLossBackward>) average train loss tensor(0.2808, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(2.2439)
epoch: 1 train_loss: tensor(0.2648, grad_fn=<NllLossBackward>) average train loss tensor(0.2388, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(2.2173)
epoch: 2 train_loss: tensor(0.2664, grad_fn=<NllLossBackward>) average train loss tensor(0.2341, grad_fn=<DivBackward0>)
val_acc: 0.025277998997138895 val_avg_loss: tensor(2.1798)
epoch: 3 train_loss: tensor(0.2613, grad_fn=<NllLossBackward>) average train loss tensor(0.2313, grad_fn=<DivBackward0>)
val_acc: 0.02775565584166593 val_avg_loss: tensor(2.2178)
epoch: 4 train_loss: tensor(0.2437, grad_fn=<NllLossBackward>) average train loss tensor(0.2281, grad_fn=<DivBackward0>)
val_acc: 0.04344748252367047 val_avg_loss: tensor(2.1501)
epoch: 5 train_loss: tensor(0.2589, grad_fn=<NllLossBackward>) average train loss tensor(0.2251, grad_

val_acc: 0.21446479662566734 val_avg_loss: tensor(1.8073)
epoch: 46 train_loss: tensor(0.1985, grad_fn=<NllLossBackward>) average train loss tensor(0.1887, grad_fn=<DivBackward0>)
val_acc: 0.22877031531132938 val_avg_loss: tensor(1.6811)
epoch: 47 train_loss: tensor(0.1881, grad_fn=<NllLossBackward>) average train loss tensor(0.1918, grad_fn=<DivBackward0>)
val_acc: 0.22517181370380202 val_avg_loss: tensor(1.8571)
epoch: 48 train_loss: tensor(0.1799, grad_fn=<NllLossBackward>) average train loss tensor(0.1896, grad_fn=<DivBackward0>)
val_acc: 0.22449340766303866 val_avg_loss: tensor(1.7792)
epoch: 49 train_loss: tensor(0.1805, grad_fn=<NllLossBackward>) average train loss tensor(0.1862, grad_fn=<DivBackward0>)
val_acc: 0.27109695307199955 val_avg_loss: tensor(1.6459)
epoch: 50 train_loss: tensor(0.1972, grad_fn=<NllLossBackward>) average train loss tensor(0.1904, grad_fn=<DivBackward0>)
val_acc: 0.2255847565112232 val_avg_loss: tensor(1.8182)
epoch: 51 train_loss: tensor(0.1888, grad_f

val_acc: 0.2836032209538979 val_avg_loss: tensor(1.7061)
epoch: 92 train_loss: tensor(0.1680, grad_fn=<NllLossBackward>) average train loss tensor(0.1750, grad_fn=<DivBackward0>)
val_acc: 0.3167566292068549 val_avg_loss: tensor(1.5680)
epoch: 93 train_loss: tensor(0.1662, grad_fn=<NllLossBackward>) average train loss tensor(0.1754, grad_fn=<DivBackward0>)
val_acc: 0.305636669321299 val_avg_loss: tensor(1.6694)
epoch: 94 train_loss: tensor(0.1699, grad_fn=<NllLossBackward>) average train loss tensor(0.1807, grad_fn=<DivBackward0>)
val_acc: 0.2905052650207946 val_avg_loss: tensor(1.6601)
epoch: 95 train_loss: tensor(0.1674, grad_fn=<NllLossBackward>) average train loss tensor(0.1772, grad_fn=<DivBackward0>)
val_acc: 0.2994130312951656 val_avg_loss: tensor(1.5759)
epoch: 96 train_loss: tensor(0.1710, grad_fn=<NllLossBackward>) average train loss tensor(0.1787, grad_fn=<DivBackward0>)
val_acc: 0.30436834498421966 val_avg_loss: tensor(1.5823)
epoch: 97 train_loss: tensor(0.1704, grad_fn=<Nl

In [56]:
model = WtsNormModelBN(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
writer = SummaryWriter('runs/norm_bn_d128_drop05_wd0005')

fit_wts_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    writer=writer,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader
)

epoch: 0 train_loss: tensor(0.2815, grad_fn=<NllLossBackward>) average train loss tensor(0.5123, grad_fn=<DivBackward0>)
val_acc: 0.9746925050880453 val_avg_loss: tensor(0.2874)
epoch: 1 train_loss: tensor(0.1039, grad_fn=<NllLossBackward>) average train loss tensor(0.1661, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1152)
epoch: 2 train_loss: tensor(0.0923, grad_fn=<NllLossBackward>) average train loss tensor(0.1213, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1122)
epoch: 3 train_loss: tensor(0.0904, grad_fn=<NllLossBackward>) average train loss tensor(0.1183, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1114)
epoch: 4 train_loss: tensor(0.0848, grad_fn=<NllLossBackward>) average train loss tensor(0.1169, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1105)
epoch: 5 train_loss: tensor(0.0903, grad_fn=<NllLossBackward>) average train loss tensor(0.1160, grad_fn=<DivB

epoch: 46 train_loss: tensor(0.0587, grad_fn=<NllLossBackward>) average train loss tensor(0.0978, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1185)
epoch: 47 train_loss: tensor(0.0645, grad_fn=<NllLossBackward>) average train loss tensor(0.0979, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1205)
epoch: 48 train_loss: tensor(0.0546, grad_fn=<NllLossBackward>) average train loss tensor(0.0971, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1205)
epoch: 49 train_loss: tensor(0.0652, grad_fn=<NllLossBackward>) average train loss tensor(0.0962, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1208)
epoch: 50 train_loss: tensor(0.0542, grad_fn=<NllLossBackward>) average train loss tensor(0.0958, grad_fn=<DivBackward0>)
val_acc: 0.9746925050880453 val_avg_loss: tensor(0.1195)
epoch: 51 train_loss: tensor(0.0546, grad_fn=<NllLossBackward>) average train loss tensor(0.0964, grad_fn

epoch: 92 train_loss: tensor(0.0409, grad_fn=<NllLossBackward>) average train loss tensor(0.0849, grad_fn=<DivBackward0>)
val_acc: 0.9746925050880453 val_avg_loss: tensor(0.1280)
epoch: 93 train_loss: tensor(0.0352, grad_fn=<NllLossBackward>) average train loss tensor(0.0839, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1268)
epoch: 94 train_loss: tensor(0.0416, grad_fn=<NllLossBackward>) average train loss tensor(0.0845, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1319)
epoch: 95 train_loss: tensor(0.0433, grad_fn=<NllLossBackward>) average train loss tensor(0.0853, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1279)
epoch: 96 train_loss: tensor(0.0420, grad_fn=<NllLossBackward>) average train loss tensor(0.0839, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1295)
epoch: 97 train_loss: tensor(0.0416, grad_fn=<NllLossBackward>) average train loss tensor(0.0839, grad_fn

In [114]:
model = WtsNormModelBN(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
writer = SummaryWriter('runs/norm_bn_d128_drop05_wd0005_weight200')

fit_wts_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    writer=writer,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader,
    weight=torch.tensor([1, 200]).float()
)

predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))
print('roc auc score:', roc_auc_score(y_wts_test, predictions))

epoch: 0 train_loss: tensor(0.6199, grad_fn=<NllLossBackward>) average train loss tensor(0.5532, grad_fn=<DivBackward0>)
val_acc: 0.22036397958882695 val_avg_loss: tensor(1.1939)
epoch: 1 train_loss: tensor(0.4589, grad_fn=<NllLossBackward>) average train loss tensor(0.4401, grad_fn=<DivBackward0>)
val_acc: 0.19414211131758252 val_avg_loss: tensor(1.3056)
epoch: 2 train_loss: tensor(0.4277, grad_fn=<NllLossBackward>) average train loss tensor(0.4055, grad_fn=<DivBackward0>)
val_acc: 0.18169483526531574 val_avg_loss: tensor(1.4024)
epoch: 3 train_loss: tensor(0.4529, grad_fn=<NllLossBackward>) average train loss tensor(0.3922, grad_fn=<DivBackward0>)
val_acc: 0.16564905760552165 val_avg_loss: tensor(1.3765)
epoch: 4 train_loss: tensor(0.4084, grad_fn=<NllLossBackward>) average train loss tensor(0.3849, grad_fn=<DivBackward0>)
val_acc: 0.20741527298469162 val_avg_loss: tensor(1.3747)
epoch: 5 train_loss: tensor(0.4195, grad_fn=<NllLossBackward>) average train loss tensor(0.3780, grad_fn=

epoch: 46 train_loss: tensor(0.2789, grad_fn=<NllLossBackward>) average train loss tensor(0.2736, grad_fn=<DivBackward0>)
val_acc: 0.5647877768929004 val_avg_loss: tensor(0.9518)
epoch: 47 train_loss: tensor(0.2872, grad_fn=<NllLossBackward>) average train loss tensor(0.2709, grad_fn=<DivBackward0>)
val_acc: 0.5649057605521636 val_avg_loss: tensor(0.9513)
epoch: 48 train_loss: tensor(0.2629, grad_fn=<NllLossBackward>) average train loss tensor(0.2683, grad_fn=<DivBackward0>)
val_acc: 0.5682977907559803 val_avg_loss: tensor(0.9404)
epoch: 49 train_loss: tensor(0.2857, grad_fn=<NllLossBackward>) average train loss tensor(0.2723, grad_fn=<DivBackward0>)
val_acc: 0.5650827360410583 val_avg_loss: tensor(0.9587)
epoch: 50 train_loss: tensor(0.2730, grad_fn=<NllLossBackward>) average train loss tensor(0.2699, grad_fn=<DivBackward0>)
val_acc: 0.5286552812435478 val_avg_loss: tensor(1.0362)
epoch: 51 train_loss: tensor(0.2648, grad_fn=<NllLossBackward>) average train loss tensor(0.2641, grad_fn

epoch: 92 train_loss: tensor(0.2442, grad_fn=<NllLossBackward>) average train loss tensor(0.2337, grad_fn=<DivBackward0>)
val_acc: 0.6581423472849011 val_avg_loss: tensor(0.7843)
epoch: 93 train_loss: tensor(0.2720, grad_fn=<NllLossBackward>) average train loss tensor(0.2377, grad_fn=<DivBackward0>)
val_acc: 0.6683184378963514 val_avg_loss: tensor(0.7566)
epoch: 94 train_loss: tensor(0.2413, grad_fn=<NllLossBackward>) average train loss tensor(0.2336, grad_fn=<DivBackward0>)
val_acc: 0.6279385305135239 val_avg_loss: tensor(0.8652)
epoch: 95 train_loss: tensor(0.2389, grad_fn=<NllLossBackward>) average train loss tensor(0.2382, grad_fn=<DivBackward0>)
val_acc: 0.6516532460254255 val_avg_loss: tensor(0.7704)
epoch: 96 train_loss: tensor(0.2243, grad_fn=<NllLossBackward>) average train loss tensor(0.2372, grad_fn=<DivBackward0>)
val_acc: 0.6644249771406661 val_avg_loss: tensor(0.7441)
epoch: 97 train_loss: tensor(0.2324, grad_fn=<NllLossBackward>) average train loss tensor(0.2341, grad_fn

In [113]:
model = WtsNormModelBN(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
writer = SummaryWriter('runs/norm_bn_d128_drop05_wd0005_weight300')

fit_wts_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    writer=writer,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader,
    weight=torch.tensor([1, 300]).float()
)

predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))
print('roc auc score:', roc_auc_score(y_wts_test, predictions))

epoch: 0 train_loss: tensor(0.6133, grad_fn=<NllLossBackward>) average train loss tensor(0.5558, grad_fn=<DivBackward0>)
val_acc: 0.26499129870512933 val_avg_loss: tensor(1.0509)
epoch: 1 train_loss: tensor(0.3602, grad_fn=<NllLossBackward>) average train loss tensor(0.3941, grad_fn=<DivBackward0>)
val_acc: 0.21319647228858804 val_avg_loss: tensor(1.2913)
epoch: 2 train_loss: tensor(0.4233, grad_fn=<NllLossBackward>) average train loss tensor(0.3425, grad_fn=<DivBackward0>)
val_acc: 0.16570804943515324 val_avg_loss: tensor(1.5446)
epoch: 3 train_loss: tensor(0.3665, grad_fn=<NllLossBackward>) average train loss tensor(0.3227, grad_fn=<DivBackward0>)
val_acc: 0.15364422027549185 val_avg_loss: tensor(1.5891)
epoch: 4 train_loss: tensor(0.3388, grad_fn=<NllLossBackward>) average train loss tensor(0.3140, grad_fn=<DivBackward0>)
val_acc: 0.17390791375394507 val_avg_loss: tensor(1.6234)
epoch: 5 train_loss: tensor(0.3409, grad_fn=<NllLossBackward>) average train loss tensor(0.3087, grad_fn=

epoch: 46 train_loss: tensor(0.2790, grad_fn=<NllLossBackward>) average train loss tensor(0.2370, grad_fn=<DivBackward0>)
val_acc: 0.43347196413296757 val_avg_loss: tensor(1.3404)
epoch: 47 train_loss: tensor(0.2455, grad_fn=<NllLossBackward>) average train loss tensor(0.2344, grad_fn=<DivBackward0>)
val_acc: 0.4466566380556293 val_avg_loss: tensor(1.3311)
epoch: 48 train_loss: tensor(0.2569, grad_fn=<NllLossBackward>) average train loss tensor(0.2337, grad_fn=<DivBackward0>)
val_acc: 0.4570097041559744 val_avg_loss: tensor(1.3299)
epoch: 49 train_loss: tensor(0.2440, grad_fn=<NllLossBackward>) average train loss tensor(0.2338, grad_fn=<DivBackward0>)
val_acc: 0.48576822110137746 val_avg_loss: tensor(1.2244)
epoch: 50 train_loss: tensor(0.2573, grad_fn=<NllLossBackward>) average train loss tensor(0.2296, grad_fn=<DivBackward0>)
val_acc: 0.45591835530778985 val_avg_loss: tensor(1.2939)
epoch: 51 train_loss: tensor(0.2469, grad_fn=<NllLossBackward>) average train loss tensor(0.2321, grad

epoch: 92 train_loss: tensor(0.2355, grad_fn=<NllLossBackward>) average train loss tensor(0.2166, grad_fn=<DivBackward0>)
val_acc: 0.5245848449989676 val_avg_loss: tensor(1.1127)
epoch: 93 train_loss: tensor(0.2121, grad_fn=<NllLossBackward>) average train loss tensor(0.2110, grad_fn=<DivBackward0>)
val_acc: 0.5469722443441584 val_avg_loss: tensor(1.0687)
epoch: 94 train_loss: tensor(0.2275, grad_fn=<NllLossBackward>) average train loss tensor(0.2148, grad_fn=<DivBackward0>)
val_acc: 0.5201309618617821 val_avg_loss: tensor(1.1133)
epoch: 95 train_loss: tensor(0.2281, grad_fn=<NllLossBackward>) average train loss tensor(0.2117, grad_fn=<DivBackward0>)
val_acc: 0.5027578680352771 val_avg_loss: tensor(1.2100)
epoch: 96 train_loss: tensor(0.2341, grad_fn=<NllLossBackward>) average train loss tensor(0.2047, grad_fn=<DivBackward0>)
val_acc: 0.5577087573371088 val_avg_loss: tensor(1.1023)
epoch: 97 train_loss: tensor(0.2371, grad_fn=<NllLossBackward>) average train loss tensor(0.2135, grad_fn

In [112]:
model = WtsNormModelBN(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
writer = SummaryWriter('runs/norm_bn_d128_drop05_wd0005_weight400')

fit_wts_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    writer=writer,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader,
    weight=torch.tensor([1, 400]).float()
)

predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))
print('roc auc score:', roc_auc_score(y_wts_test, predictions))

epoch: 0 train_loss: tensor(0.4619, grad_fn=<NllLossBackward>) average train loss tensor(0.5538, grad_fn=<DivBackward0>)
val_acc: 0.29622747249505943 val_avg_loss: tensor(1.0865)
epoch: 1 train_loss: tensor(0.3213, grad_fn=<NllLossBackward>) average train loss tensor(0.3647, grad_fn=<DivBackward0>)
val_acc: 0.18951125269150224 val_avg_loss: tensor(1.4988)
epoch: 2 train_loss: tensor(0.3050, grad_fn=<NllLossBackward>) average train loss tensor(0.2968, grad_fn=<DivBackward0>)
val_acc: 0.12379435448190426 val_avg_loss: tensor(1.5583)
epoch: 3 train_loss: tensor(0.3108, grad_fn=<NllLossBackward>) average train loss tensor(0.2752, grad_fn=<DivBackward0>)
val_acc: 0.12553461345603634 val_avg_loss: tensor(1.8261)
epoch: 4 train_loss: tensor(0.3124, grad_fn=<NllLossBackward>) average train loss tensor(0.2623, grad_fn=<DivBackward0>)
val_acc: 0.12818924578945817 val_avg_loss: tensor(1.8401)
epoch: 5 train_loss: tensor(0.3076, grad_fn=<NllLossBackward>) average train loss tensor(0.2573, grad_fn=

epoch: 46 train_loss: tensor(0.2219, grad_fn=<NllLossBackward>) average train loss tensor(0.2047, grad_fn=<DivBackward0>)
val_acc: 0.3608530218564729 val_avg_loss: tensor(1.5872)
epoch: 47 train_loss: tensor(0.2356, grad_fn=<NllLossBackward>) average train loss tensor(0.2050, grad_fn=<DivBackward0>)
val_acc: 0.36858095153821196 val_avg_loss: tensor(1.5618)
epoch: 48 train_loss: tensor(0.2168, grad_fn=<NllLossBackward>) average train loss tensor(0.2014, grad_fn=<DivBackward0>)
val_acc: 0.4046249594431171 val_avg_loss: tensor(1.4709)
epoch: 49 train_loss: tensor(0.2298, grad_fn=<NllLossBackward>) average train loss tensor(0.2060, grad_fn=<DivBackward0>)
val_acc: 0.3511783617968911 val_avg_loss: tensor(1.6068)
epoch: 50 train_loss: tensor(0.2251, grad_fn=<NllLossBackward>) average train loss tensor(0.1997, grad_fn=<DivBackward0>)
val_acc: 0.40294369229861665 val_avg_loss: tensor(1.5399)
epoch: 51 train_loss: tensor(0.2307, grad_fn=<NllLossBackward>) average train loss tensor(0.2046, grad_

epoch: 92 train_loss: tensor(0.2083, grad_fn=<NllLossBackward>) average train loss tensor(0.1885, grad_fn=<DivBackward0>)
val_acc: 0.44627319116302394 val_avg_loss: tensor(1.4210)
epoch: 93 train_loss: tensor(0.2093, grad_fn=<NllLossBackward>) average train loss tensor(0.1894, grad_fn=<DivBackward0>)
val_acc: 0.4287231218476241 val_avg_loss: tensor(1.4428)
epoch: 94 train_loss: tensor(0.2380, grad_fn=<NllLossBackward>) average train loss tensor(0.1920, grad_fn=<DivBackward0>)
val_acc: 0.4514349762557886 val_avg_loss: tensor(1.3781)
epoch: 95 train_loss: tensor(0.2208, grad_fn=<NllLossBackward>) average train loss tensor(0.1918, grad_fn=<DivBackward0>)
val_acc: 0.40671916939503877 val_avg_loss: tensor(1.5034)
epoch: 96 train_loss: tensor(0.2037, grad_fn=<NllLossBackward>) average train loss tensor(0.1912, grad_fn=<DivBackward0>)
val_acc: 0.43400289059965197 val_avg_loss: tensor(1.4004)
epoch: 97 train_loss: tensor(0.2119, grad_fn=<NllLossBackward>) average train loss tensor(0.1895, grad

In [68]:
model = WtsTridentModelBN(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
writer = SummaryWriter('runs/trident_bn_d128_drop05_wd0005')

fit_wts_trident_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    writer=writer,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader
)

epoch: 0 train_loss: tensor(1.1756, grad_fn=<AddBackward0>) average train loss tensor(1.7794, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.9943)
epoch: 1 train_loss: tensor(0.3762, grad_fn=<AddBackward0>) average train loss tensor(0.6927, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.3632)
epoch: 2 train_loss: tensor(0.3254, grad_fn=<AddBackward0>) average train loss tensor(0.4276, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.3501)
epoch: 3 train_loss: tensor(0.2988, grad_fn=<AddBackward0>) average train loss tensor(0.3949, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.3456)
epoch: 4 train_loss: tensor(0.2754, grad_fn=<AddBackward0>) average train loss tensor(0.3815, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.3440)
epoch: 5 train_loss: tensor(0.2770, grad_fn=<AddBackward0>) average train loss tensor(0.3707, grad_fn=<DivBackward0>)
val_acc

KeyboardInterrupt: 

In [81]:
predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())[0]
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

In [88]:
print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99     41251
           1       0.00      0.00      0.00      1128

    accuracy                           0.97     42379
   macro avg       0.49      0.50      0.49     42379
weighted avg       0.95      0.97      0.96     42379



In [130]:
model = WtsTridentModelBN(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
writer = SummaryWriter('runs/trident_bn_d128_drop05_wd0005_weight200')

fit_wts_trident_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    writer=writer,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader,
    weight=torch.tensor([1, 200]).float()
)

predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())[0]
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))
print('roc auc score:', roc_auc_score(y_wts_test, predictions))

epoch: 0 train_loss: tensor(1.6483, grad_fn=<AddBackward0>) average train loss tensor(1.7840, grad_fn=<DivBackward0>)
val_acc: 0.19316874612866117 val_avg_loss: tensor(3.3955)
epoch: 1 train_loss: tensor(1.4027, grad_fn=<AddBackward0>) average train loss tensor(1.3853, grad_fn=<DivBackward0>)
val_acc: 0.17346547503170812 val_avg_loss: tensor(4.0631)
epoch: 2 train_loss: tensor(1.3353, grad_fn=<AddBackward0>) average train loss tensor(1.2676, grad_fn=<DivBackward0>)
val_acc: 0.17086983452791787 val_avg_loss: tensor(4.4315)
epoch: 3 train_loss: tensor(1.2844, grad_fn=<AddBackward0>) average train loss tensor(1.2223, grad_fn=<DivBackward0>)
val_acc: 0.1766805297466301 val_avg_loss: tensor(4.5140)
epoch: 4 train_loss: tensor(1.2315, grad_fn=<AddBackward0>) average train loss tensor(1.1869, grad_fn=<DivBackward0>)
val_acc: 0.1853523287024747 val_avg_loss: tensor(4.5919)
epoch: 5 train_loss: tensor(1.2346, grad_fn=<AddBackward0>) average train loss tensor(1.1656, grad_fn=<DivBackward0>)
val_

val_acc: 0.6000648910125947 val_avg_loss: tensor(3.3113)
epoch: 47 train_loss: tensor(0.7556, grad_fn=<AddBackward0>) average train loss tensor(0.7751, grad_fn=<DivBackward0>)
val_acc: 0.6251364186060231 val_avg_loss: tensor(3.2216)
epoch: 48 train_loss: tensor(0.7554, grad_fn=<AddBackward0>) average train loss tensor(0.7740, grad_fn=<DivBackward0>)
val_acc: 0.6158747013538625 val_avg_loss: tensor(3.2017)
epoch: 49 train_loss: tensor(0.7420, grad_fn=<AddBackward0>) average train loss tensor(0.7759, grad_fn=<DivBackward0>)
val_acc: 0.6265522225171813 val_avg_loss: tensor(3.2072)
epoch: 50 train_loss: tensor(0.7664, grad_fn=<AddBackward0>) average train loss tensor(0.7724, grad_fn=<DivBackward0>)
val_acc: 0.6014511990089373 val_avg_loss: tensor(3.2505)
epoch: 51 train_loss: tensor(0.7814, grad_fn=<AddBackward0>) average train loss tensor(0.7786, grad_fn=<DivBackward0>)
val_acc: 0.6099755183907029 val_avg_loss: tensor(3.1696)
epoch: 52 train_loss: tensor(0.7624, grad_fn=<AddBackward0>) av

val_acc: 0.7022682358493348 val_avg_loss: tensor(2.7702)
epoch: 94 train_loss: tensor(0.6671, grad_fn=<AddBackward0>) average train loss tensor(0.6745, grad_fn=<DivBackward0>)
val_acc: 0.716986697342418 val_avg_loss: tensor(2.7569)
epoch: 95 train_loss: tensor(0.6101, grad_fn=<AddBackward0>) average train loss tensor(0.6611, grad_fn=<DivBackward0>)
val_acc: 0.7428546146358729 val_avg_loss: tensor(2.6305)
epoch: 96 train_loss: tensor(0.6405, grad_fn=<AddBackward0>) average train loss tensor(0.6584, grad_fn=<DivBackward0>)
val_acc: 0.6811786567560393 val_avg_loss: tensor(2.8386)
epoch: 97 train_loss: tensor(0.6300, grad_fn=<AddBackward0>) average train loss tensor(0.6482, grad_fn=<DivBackward0>)
val_acc: 0.7322950771318173 val_avg_loss: tensor(2.5888)
epoch: 98 train_loss: tensor(0.6182, grad_fn=<AddBackward0>) average train loss tensor(0.6508, grad_fn=<DivBackward0>)
val_acc: 0.7079609474087839 val_avg_loss: tensor(2.6607)
epoch: 99 train_loss: tensor(0.6632, grad_fn=<AddBackward0>) ave

In [127]:
model = WtsTridentModelBN(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
writer = SummaryWriter('runs/trident_bn_d128_drop05_wd0005_weight300')

fit_wts_trident_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    writer=writer,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader,
    weight=torch.tensor([1, 300]).float()
)



epoch: 0 train_loss: tensor(1.6898, grad_fn=<AddBackward0>) average train loss tensor(1.7415, grad_fn=<DivBackward0>)
val_acc: 0.23531840840043655 val_avg_loss: tensor(3.4458)
epoch: 1 train_loss: tensor(1.0159, grad_fn=<AddBackward0>) average train loss tensor(1.2337, grad_fn=<DivBackward0>)
val_acc: 0.15500103235701856 val_avg_loss: tensor(4.3456)
epoch: 2 train_loss: tensor(1.1144, grad_fn=<AddBackward0>) average train loss tensor(1.0647, grad_fn=<DivBackward0>)
val_acc: 0.13759844261569773 val_avg_loss: tensor(4.9705)
epoch: 3 train_loss: tensor(1.0784, grad_fn=<AddBackward0>) average train loss tensor(0.9980, grad_fn=<DivBackward0>)
val_acc: 0.14588679467893698 val_avg_loss: tensor(5.3533)
epoch: 4 train_loss: tensor(1.0693, grad_fn=<AddBackward0>) average train loss tensor(0.9671, grad_fn=<DivBackward0>)
val_acc: 0.15155001032357018 val_avg_loss: tensor(5.3229)
epoch: 5 train_loss: tensor(0.9872, grad_fn=<AddBackward0>) average train loss tensor(0.9408, grad_fn=<DivBackward0>)
va

val_acc: 0.5571483349556087 val_avg_loss: tensor(3.9324)
epoch: 47 train_loss: tensor(0.6967, grad_fn=<AddBackward0>) average train loss tensor(0.6806, grad_fn=<DivBackward0>)
val_acc: 0.5306610034510221 val_avg_loss: tensor(4.0804)
epoch: 48 train_loss: tensor(0.6772, grad_fn=<AddBackward0>) average train loss tensor(0.6774, grad_fn=<DivBackward0>)
val_acc: 0.5728401616376132 val_avg_loss: tensor(3.9678)
epoch: 49 train_loss: tensor(0.7497, grad_fn=<AddBackward0>) average train loss tensor(0.6727, grad_fn=<DivBackward0>)
val_acc: 0.5106332772910952 val_avg_loss: tensor(4.2124)
epoch: 50 train_loss: tensor(0.6878, grad_fn=<AddBackward0>) average train loss tensor(0.6665, grad_fn=<DivBackward0>)
val_acc: 0.5612482671150045 val_avg_loss: tensor(3.8997)
epoch: 51 train_loss: tensor(0.6772, grad_fn=<AddBackward0>) average train loss tensor(0.6673, grad_fn=<DivBackward0>)
val_acc: 0.5364422027549185 val_avg_loss: tensor(3.9579)
epoch: 52 train_loss: tensor(0.7034, grad_fn=<AddBackward0>) av

val_acc: 0.5851989499454325 val_avg_loss: tensor(3.6917)
epoch: 94 train_loss: tensor(0.5775, grad_fn=<AddBackward0>) average train loss tensor(0.5896, grad_fn=<DivBackward0>)
val_acc: 0.6729198006076158 val_avg_loss: tensor(3.2720)
epoch: 95 train_loss: tensor(0.6035, grad_fn=<AddBackward0>) average train loss tensor(0.5711, grad_fn=<DivBackward0>)
val_acc: 0.649971978880925 val_avg_loss: tensor(3.4860)
epoch: 96 train_loss: tensor(0.6269, grad_fn=<AddBackward0>) average train loss tensor(0.5742, grad_fn=<DivBackward0>)
val_acc: 0.6455770875733711 val_avg_loss: tensor(3.4098)
epoch: 97 train_loss: tensor(0.5823, grad_fn=<AddBackward0>) average train loss tensor(0.5892, grad_fn=<DivBackward0>)
val_acc: 0.6436598531103442 val_avg_loss: tensor(3.4043)
epoch: 98 train_loss: tensor(0.5887, grad_fn=<AddBackward0>) average train loss tensor(0.5942, grad_fn=<DivBackward0>)
val_acc: 0.6188537887502581 val_avg_loss: tensor(3.5111)
epoch: 99 train_loss: tensor(0.6054, grad_fn=<AddBackward0>) ave

TypeError: expected Tensor as element 1 in argument 0, but got tuple

In [128]:
predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())[0]
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))
print('roc auc score:', roc_auc_score(y_wts_test, predictions))

              precision    recall  f1-score   support

           0       0.98      0.64      0.78     41251
           1       0.04      0.60      0.08      1128

    accuracy                           0.64     42379
   macro avg       0.51      0.62      0.43     42379
weighted avg       0.96      0.64      0.76     42379

roc auc score: 0.6685758348690795


In [129]:
model = WtsTridentModelBN(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
writer = SummaryWriter('runs/trident_bn_d128_drop05_wd0005_weight400')

fit_wts_trident_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    writer=writer,
    train_loader=wts_train_loader,
    val_loader=wts_val_loader,
    weight=torch.tensor([1, 400]).float()
)

predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())[0]
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))
print('roc auc score:', roc_auc_score(y_wts_test, predictions))

epoch: 0 train_loss: tensor(1.2018, grad_fn=<AddBackward0>) average train loss tensor(1.6803, grad_fn=<DivBackward0>)
val_acc: 0.20968645842550807 val_avg_loss: tensor(3.7209)
epoch: 1 train_loss: tensor(1.0303, grad_fn=<AddBackward0>) average train loss tensor(1.1339, grad_fn=<DivBackward0>)
val_acc: 0.1824912249653423 val_avg_loss: tensor(4.7416)
epoch: 2 train_loss: tensor(0.9356, grad_fn=<AddBackward0>) average train loss tensor(0.9283, grad_fn=<DivBackward0>)
val_acc: 0.12290947703743031 val_avg_loss: tensor(5.6319)
epoch: 3 train_loss: tensor(0.9425, grad_fn=<AddBackward0>) average train loss tensor(0.8664, grad_fn=<DivBackward0>)
val_acc: 0.11686281450019172 val_avg_loss: tensor(6.0184)
epoch: 4 train_loss: tensor(0.9064, grad_fn=<AddBackward0>) average train loss tensor(0.8272, grad_fn=<DivBackward0>)
val_acc: 0.13535675308969708 val_avg_loss: tensor(6.0717)
epoch: 5 train_loss: tensor(0.9601, grad_fn=<AddBackward0>) average train loss tensor(0.8115, grad_fn=<DivBackward0>)
val

val_acc: 0.4629383830339498 val_avg_loss: tensor(4.9602)
epoch: 47 train_loss: tensor(0.6525, grad_fn=<AddBackward0>) average train loss tensor(0.5966, grad_fn=<DivBackward0>)
val_acc: 0.43827979824794266 val_avg_loss: tensor(5.0094)
epoch: 48 train_loss: tensor(0.6674, grad_fn=<AddBackward0>) average train loss tensor(0.5943, grad_fn=<DivBackward0>)
val_acc: 0.4433530955962599 val_avg_loss: tensor(4.9712)
epoch: 49 train_loss: tensor(0.6685, grad_fn=<AddBackward0>) average train loss tensor(0.5995, grad_fn=<DivBackward0>)
val_acc: 0.4448278913370498 val_avg_loss: tensor(4.9863)
epoch: 50 train_loss: tensor(0.6253, grad_fn=<AddBackward0>) average train loss tensor(0.6000, grad_fn=<DivBackward0>)
val_acc: 0.4640887237117659 val_avg_loss: tensor(4.8778)
epoch: 51 train_loss: tensor(0.6273, grad_fn=<AddBackward0>) average train loss tensor(0.5844, grad_fn=<DivBackward0>)
val_acc: 0.4732914491342949 val_avg_loss: tensor(4.8611)
epoch: 52 train_loss: tensor(0.6782, grad_fn=<AddBackward0>) a

val_acc: 0.5574727900185824 val_avg_loss: tensor(4.2845)
epoch: 94 train_loss: tensor(0.5774, grad_fn=<AddBackward0>) average train loss tensor(0.5413, grad_fn=<DivBackward0>)
val_acc: 0.5497153644220275 val_avg_loss: tensor(4.3484)
epoch: 95 train_loss: tensor(0.5768, grad_fn=<AddBackward0>) average train loss tensor(0.5388, grad_fn=<DivBackward0>)
val_acc: 0.5319588237029171 val_avg_loss: tensor(4.4421)
epoch: 96 train_loss: tensor(0.5621, grad_fn=<AddBackward0>) average train loss tensor(0.5326, grad_fn=<DivBackward0>)
val_acc: 0.5320473114473645 val_avg_loss: tensor(4.3767)
epoch: 97 train_loss: tensor(0.5594, grad_fn=<AddBackward0>) average train loss tensor(0.5393, grad_fn=<DivBackward0>)
val_acc: 0.5351738784178391 val_avg_loss: tensor(4.4216)
epoch: 98 train_loss: tensor(0.5520, grad_fn=<AddBackward0>) average train loss tensor(0.5338, grad_fn=<DivBackward0>)
val_acc: 0.5740199982302451 val_avg_loss: tensor(4.2358)
epoch: 99 train_loss: tensor(0.5811, grad_fn=<AddBackward0>) av

## Upsample 

In [119]:
def get_upsample_worthiness_loader(upsample_rate=1):
    indices_0 = (y_wts_train[:, 1] == 0).nonzero()[0]
    indices_1 = (y_wts_train[:, 1] == 1).nonzero()[0]
    
    n0 = (y_wts_train[:, 1] == 0).nonzero()[0].shape[0]
    n1 = (y_wts_train[:, 1] == 1).nonzero()[0].shape[0]
    
    upsample_indices = indices_1[np.random.choice(n1, n0 * upsample_rate)]
    concatenated_indices = np.concatenate((indices_0, upsample_indices))
    shuffled_indices = np.random.permutation(concatenated_indices)

    x_img_train_upsampled = x_img_train[shuffled_indices]
    x_txt_train_upsampled = x_txt_train[shuffled_indices]
    y_wts_train_upsampled = y_wts_train[shuffled_indices]
    
    x_img_train_upsampled_t = torch.tensor(x_img_train_upsampled).float()
    x_txt_train_upsampled_t = torch.tensor(x_txt_train_upsampled).float()
    y_wts_train_upsampled_t = torch.tensor(y_wts_train_upsampled).float()
 
    train_ds_upsampled = TensorDataset(
        x_img_train_upsampled_t, 
        x_txt_train_upsampled_t,
        y_wts_train_upsampled_t
    )
    
    return DataLoader(train_ds_upsampled, batch_size=BATCH_SIZE)

In [120]:
for upsample_rate in range(3):
    print('train with upsample_rate', upsample_rate)
    
    model = WtsNormModelBN(d=128, drop=0.5)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
    writer = SummaryWriter('runs/norm_d128_drop05_wd0005_upsample' + str(upsample_rate))

    fit_wts_model(
        model=model,
        optimizer=optimizer,
        epochs=100,
        writer=writer,
        train_loader=get_upsample_worthiness_loader(upsample_rate=upsample_rate),
        val_loader=wts_val_loader
    )
    
    predictions = torch.tensor([])

    with torch.no_grad():
        for x_img_cur, x_txt_cur, _ in wts_test_loader:
            outputs = model(x_img_cur.float(), x_txt_cur.float())
            predictions = torch.cat((predictions, outputs), 0)

    predictions = predictions.numpy()

    print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))

train with upsample_rate 0
epoch: 0 train_loss: tensor(0.2462, grad_fn=<NllLossBackward>) average train loss tensor(0.5391, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.2293)
epoch: 1 train_loss: tensor(0.0280, grad_fn=<NllLossBackward>) average train loss tensor(0.0895, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1176)
epoch: 2 train_loss: tensor(0.0104, grad_fn=<NllLossBackward>) average train loss tensor(0.0167, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1292)
epoch: 3 train_loss: tensor(0.0057, grad_fn=<NllLossBackward>) average train loss tensor(0.0076, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1405)
epoch: 4 train_loss: tensor(0.0037, grad_fn=<NllLossBackward>) average train loss tensor(0.0045, grad_fn=<DivBackward0>)
val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1497)
epoch: 5 train_loss: tensor(0.0026, grad_fn=<NllLossBackward>) average train loss t

val_acc: 0.9747220010028611 val_avg_loss: tensor(0.1988)


KeyboardInterrupt: 

Classification report for norm model fitted on upsampled ds with upsample_rate=1

In [121]:
predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99     41251
           1       0.00      0.00      0.00      1128

    accuracy                           0.97     42379
   macro avg       0.49      0.50      0.49     42379
weighted avg       0.95      0.97      0.96     42379



/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [122]:
model = WtsNormModelBN(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
writer = SummaryWriter('runs/norm_d128_drop05_wd0005_upsample2')

fit_wts_model(
    model=model,
    optimizer=optimizer,
    epochs=40,
    writer=writer,
    train_loader=get_upsample_worthiness_loader(upsample_rate=2),
    val_loader=wts_val_loader
)
    
predictions = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _ in wts_test_loader:
        outputs = model(x_img_cur.float(), x_txt_cur.float())
        predictions = torch.cat((predictions, outputs), 0)

predictions = predictions.numpy()

print(classification_report(np.argmax(y_wts_test, axis=1), np.argmax(predictions, axis=1)))

epoch: 0 train_loss: tensor(0.3790, grad_fn=<NllLossBackward>) average train loss tensor(0.5243, grad_fn=<DivBackward0>)
val_acc: 0.6273191163023921 val_avg_loss: tensor(0.7520)
epoch: 1 train_loss: tensor(0.2378, grad_fn=<NllLossBackward>) average train loss tensor(0.3125, grad_fn=<DivBackward0>)
val_acc: 0.8132908592159985 val_avg_loss: tensor(0.4452)
epoch: 2 train_loss: tensor(0.1782, grad_fn=<NllLossBackward>) average train loss tensor(0.2118, grad_fn=<DivBackward0>)
val_acc: 0.8644662714214081 val_avg_loss: tensor(0.3616)
epoch: 3 train_loss: tensor(0.1123, grad_fn=<NllLossBackward>) average train loss tensor(0.1618, grad_fn=<DivBackward0>)
val_acc: 0.8943456331298115 val_avg_loss: tensor(0.3110)
epoch: 4 train_loss: tensor(0.1100, grad_fn=<NllLossBackward>) average train loss tensor(0.1328, grad_fn=<DivBackward0>)
val_acc: 0.9101849393858951 val_avg_loss: tensor(0.2810)
epoch: 5 train_loss: tensor(0.0857, grad_fn=<NllLossBackward>) average train loss tensor(0.1151, grad_fn=<DivB

In [123]:
print('roc auc score:', roc_auc_score(y_wts_test, predictions))

roc auc score: 0.6079165327777999
